In [1]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts

/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))
/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package yasa is out of date. Your version is 0.6.3, the latest is 0.6.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
folder = '/user/home/ko20929/work/RBD_using_custom_package/Blue_pebble/'

p_id_to_drop = []

paths = joblib.load(folder + 'Wake' + '_paths.pkl')
wake_groups = pd.Series(paths['s_p_id'])
wake_groups_counts = wake_groups.value_counts()

for ind, val in zip(wake_groups_counts.index, wake_groups_counts):
    #loop through the groups of each data type
    # print('Outer index is: ' + str(ind) ) for debugging
    for data_type in ['N2', 'N3','REM', 'N1']:
        paths = joblib.load(folder + data_type + '_paths.pkl')
        groups = pd.Series(paths['s_p_id'])
 
        if ind in groups.values: 
            pass 
        else:
            print(ind)
            p_id_to_drop.append(ind)
            wake_groups_counts = wake_groups_counts.drop(index = ind)
            break

4411
3305
1114
1115
2201


In [3]:
p_id_to_drop

['4411', '3305', '1114', '1115', '2201']

In [4]:
wake_multiple_pids = wake_groups_counts[wake_groups_counts > 1]

single_pids = []

for ind, val in zip(wake_multiple_pids.index, wake_multiple_pids):
    # For each p_id that has more than one value check if it has more than one record in other data types
    
    for data_type in ['N2', 'N3','REM', 'N1']:
        paths = joblib.load(folder + data_type + '_paths.pkl')
        groups = pd.Series(paths['s_p_id'])
        group_counts = groups.value_counts()
        
        if group_counts[ind] > 1: 
            pass
            
        else:
            single_pids.append(ind)
            
single_pids = list(np.unique(single_pids))
 

print(single_pids)

['3306', '4404']


#### Load Wake Data , drop relevant p_ids

In [5]:
#Load wake data _______________________________________________
paths = joblib.load(folder + 'Wake' + '_paths.pkl')
X = joblib.load('Wake_frequency_data.pkl')

#Load groups and y labels _____________________________________
groups = pd.Series(paths['s_p_id'])
s_class_list = pd.Series(paths['s_class_list'])
y = s_class_list.map({'HC': 0 , 'PD' : 1 , 'PD+RBD' : 2 , 'RBD' : 3}) 

#Drop p_ids that are not there in all other data types
X['groups'] = groups 
X['y'] = y
X = X[~X['groups'].isin(p_id_to_drop)]

#Drop p_ids (two) that should only be present once
for val in single_pids:
    duplicates = X[X['groups'].duplicated(keep=False) & (X['groups'] == val)]
    X = X.drop(duplicates.index[0])
# X is what I want now, all features plus additional columns called 'groups' and 'y' 
X_wake = X.copy()

In [6]:
two_ind_vals = list(X_wake['groups'].value_counts()[ (X_wake['groups'].value_counts() > 1 ) ].index)
two_ind_vals

['3304', '2211', '2204', '4401']

In [7]:
X_wake['y'].value_counts()

y
3    17
0    16
1    10
2     6
Name: count, dtype: int64

##### Join Non-Wake Data to Wake Data

In [8]:
X_N1 = joblib.load('N1_frequency_data.pkl')
paths = joblib.load(folder + 'N1' + '_paths.pkl')

groups = pd.Series(paths['s_p_id'])
s_class_list = pd.Series(paths['s_class_list'])
y = s_class_list.map({'HC': 0 , 'PD' : 1 , 'PD+RBD' : 2 , 'RBD' : 3}) 

X_N1['groups'] = groups 
X_N1['y'] = y

X_N1_groups_reduced = X_N1['groups'][X_N1['groups'].isin(list(X_wake['groups'].values)) ]

In [10]:
matching_indices = []
series = X_N1_groups_reduced
counter = 0
for val in np.unique(X_wake['groups']):
    if val in two_ind_vals:
        index_of_value = series[series == val].index[0]
        matching_indices.append(index_of_value)
        index_of_value = series[series == val].index[1]
        matching_indices.append(index_of_value)
        
    else:
        index_of_value = series[series == val].index[0]
        matching_indices.append(index_of_value)
        
len(matching_indices)


49

In [17]:
X_N1 = X_N1.loc[matching_indices]
X_N1 = X_N1.sort_values(by ='groups')
X_N1

,Delta_Central,Theta_Central,Low_Alpha_Central,Alpha_Central,High_Alpha_Central,Sigma_Central,Beta_Central,Gamma_Central,TotalAbsPow_Central,Fast_ratio_Central,...,Alpha_Right Temporal,High_Alpha_Right Temporal,Sigma_Right Temporal,Beta_Right Temporal,Gamma_Right Temporal,TotalAbsPow_Right Temporal,Fast_ratio_Right Temporal,Alpha_shift_Right Temporal,groups,y
0,0.361374,0.278515,0.217975,0.142088,0.034991,0.082963,0.106511,0.028548,30.670926,0.390253,...,0.164842,0.029249,0.070579,0.084403,0.015991,34.465101,0.375787,9.602886,1101,0
2,0.412212,0.323588,0.292569,0.156997,0.023152,0.049676,0.049705,0.007823,30.410397,0.281902,...,0.155814,0.021329,0.035768,0.042968,0.013442,13.857836,0.265113,13.761902,1102,0
3,0.486820,0.242581,0.191805,0.148365,0.028025,0.050508,0.059980,0.011746,28.830325,0.287666,...,0.176632,0.033110,0.039753,0.038475,0.006526,41.208645,0.292367,7.635879,1103,0
5,0.559781,0.173497,0.091747,0.100920,0.025555,0.056328,0.096803,0.012672,25.671308,0.271484,...,0.145814,0.038170,0.067219,0.096066,0.019768,10.900453,0.363086,2.730993,1104,0
6,0.421909,0.153595,0.127525,0.171076,0.039760,0.106347,0.133228,0.013844,20.678019,0.529953,...,0.163103,0.035062,0.066741,0.103459,0.037167,23.548554,0.426357,4.235561,1105,0
8,0.549439,0.221487,0.163947,0.103743,0.020867,0.049256,0.064732,0.011343,15.267023,0.219548,...,0.133932,0.025429,0.056400,0.080181,0.025847,16.720984,0.304801,8.023344,1106,0
10,0.457793,0.128930,0.092771,0.117146,0.028603,0.104987,0.167648,0.023496,12.309761,0.486426,...,0.149001,0.036477,0.095129,0.140034,0.035994,14.356083,0.507297,2.890017,1108,0
12,0.488969,0.168572,0.145849,0.170153,0.033773,0.055499,0.104040,0.012767,20.188670,0.418582,...,0.229816,0.041804,0.061672,0.098882,0.017240,10.660423,0.559303,4.112642,1109,0
13,0.421759,0.249018,0.210080,0.157755,0.027352,0.057495,0.101802,0.012170,23.525554,0.387458,...,0.167394,0.031396,0.060524,0.079886,0.012813,31.109801,0.364309,6.475080,1110,0
14,0.627286,0.153710,0.096957,0.093786,0.018475,0.043045,0.066875,0.015298,21.138359,0.208231,...,0.156089,0.027115,0.043509,0.069055,0.021353,16.593037,0.317713,5.713583,1111,0


In [11]:
len(matching_indices)

49